<a href="https://colab.research.google.com/github/marcosmedvescig/thesis_survival_models_for_predicting_churn/blob/master/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model selection



## Install libraries

Note: After installing the libraries for the first time restart the notebook for the new versions of the library

In [4]:
## Install libraries
!pip install pysurvival=='0.1.2'
!pip install scikit-survival=='0.13.1'
!pip install osqp=='0.5.0'

## Custom Functions

In [2]:
import numpy as np
import pandas as pd
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
from sksurv.metrics import cumulative_dynamic_auc

def evaluate_model(model,X_train,T_train, E_train,X_test,T_test,E_test,
                   c_index=True,ibs=True,c_auc=True):

    # c-index
    if c_index:
        from pysurvival.utils.metrics import concordance_index
        c_index = concordance_index(model, X_test, T_test, E_test)
        print('c-index: {0:5.4f}'.format(c_index))
    else:
        c_index=0

    # ibs
    if ibs:        
        from pysurvival.utils.display import integrated_brier_score
        ibs = integrated_brier_score(model, X_test, T_test, E_test, t_max=200,
                    figure_size=(20, 6.5) )
        print('IBS: {0:5.4f}'.format(ibs))
    else:
        ibs=1

    # cumulative_auc
    if c_auc:
        from sksurv.metrics import cumulative_dynamic_auc

        train = np.array([(e,t) for e,t in zip(E_train,T_train)],dtype=[('event', 'bool_'),('time','int_')])
        test = np.array([(e,t) for e,t in zip(E_test,T_test)],dtype=[('event', 'bool_'),('time','int_')])

        # auc does not support inf risk so we replace with a really large value
        risk = model.predict_risk(X_test)
        risk = np.where(risk == np.inf,100,risk)

        auc_time_list,mean_auc = cumulative_dynamic_auc(train, test, risk, [100,150,300,500], tied_tol=1e-08)
        print('AUC: {0:5.4f}'.format(mean_auc))
    else:
        mean_auc=0

    #return results
    results = pd.DataFrame({'c_index':[c_index],
                                'ibs':[ibs],
                                'mean_auc':[mean_auc]})
    return results

## Load Data

In [ ]:
import pandas as pd
import numpy as np
import os

# Open dataset
raw_data = pd.read_csv('https://github.com/marcosmedvescig/thesis_survival_models_for_predicting_churn/raw/master/churn_data_anonymized_20200718.csv')
raw_data.rename(columns={'survival_days': 'time', 'status': 'event'},inplace=True)

raw_data.head()

## Create Train, Test and Eval datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Remove observations according to EDA

raw_data = raw_data[raw_data['time']>=0]
raw_data = raw_data[raw_data['products_created'] <= (raw_data['products_created'].mean() + 3 * raw_data['products_created'].std())]
raw_data = raw_data[raw_data['admin_visits'] <= (raw_data['admin_visits'].mean() + 3 * raw_data['admin_visits'].std())]
raw_data = raw_data[raw_data['tx'] <= (raw_data['tx'].mean() + 3 * raw_data['tx'].std())]
raw_data = raw_data[raw_data['gmv_usd'] <= (raw_data['gmv_usd'].mean() + 3 * raw_data['gmv_usd'].std())]

raw_data.reset_index(drop=True,inplace=True)

# Defining the features

X = pd.get_dummies(raw_data.drop(['store_id','time', 'event'], axis=1))
T = raw_data['time']
E = raw_data['event']

## Create evaluation set, 70% of the raw_data.
index_train_test, index_eval = train_test_split( range(len(raw_data)), test_size = 0.7, random_state = 2020)

# Creating the X, T and E input
X_train_test = X.loc[index_train_test].reset_index( drop = True )
X_eval  = X.loc[index_eval].reset_index( drop = True )

T_train_test = T.loc[index_train_test].reset_index( drop = True )
T_eval  = T.loc[index_eval].reset_index( drop = True )

E_train_test = E.loc[index_train_test].reset_index( drop = True )
E_eval  = E.loc[index_eval].reset_index( drop = True )


## Create train and test set, 30% of the raw_data.
index_train, index_test = train_test_split( range(len(X_train_test)), test_size = 0.25, random_state = 2020)

# Creating the X, T and E input
X_train = X_train_test.loc[index_train].reset_index( drop = True )
X_test  = X_train_test.loc[index_test].reset_index( drop = True )

T_train = T_train_test.loc[index_train].reset_index( drop = True )
T_test  = T_train_test.loc[index_test].reset_index( drop = True )

E_train = E_train_test.loc[index_train].reset_index( drop = True )
E_test  = E_train_test.loc[index_test].reset_index( drop = True )

## Model Selection

In [ ]:
import pandas as pd
# Create empty dataframe for storing the results

results = pd.DataFrame()

### Standard CoxPH model

In [ ]:
from pysurvival.models.semi_parametric import CoxPHModel
import pandas as pd

# Creating an instance of the Cox PH model and fitting the data.

## Build the model
coxph = CoxPHModel()
coxph.fit(X_train, T_train, E_train, lr=0.5, l2_reg=1e-2, init_method='zeros')

## Evaluate model
tmp_results = evaluate_model(coxph,X_train,T_train,E_train,X_test,T_test,E_test)
tmp_results['model'] = ['coxph']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### DeepSurv/Non-Linear CoxPH model

In [ ]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel
import pandas as pd

# Creating an instance of the NonLinear CoxPH model and fitting the data.

### Defining the MLP structure. Here we will build a 1-hidden layer 
### with 150 units and `BentIdentity` as its activation function
structure = [ {'activation': 'BentIdentity', 'num_units': 150},  ]

## Build the model
nonlinear_coxph = NonLinearCoxPHModel(structure=structure)
nonlinear_coxph.fit(X_train, T_train, E_train, lr=1e-3, init_method='xav_uniform')

## Evaluate model
tmp_results = evaluate_model(nonlinear_coxph,X_train,T_train,E_train,X_test,T_test,E_test)
tmp_results['model'] = ['nonlinear_coxph']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### Linear MTLR model

In [ ]:
from pysurvival.models.multi_task import LinearMultiTaskModel
import pandas as pd

# Creating an instance of the Linear MTLR model and fitting the data.

## Build the model
l_mtlr = LinearMultiTaskModel(bins=50)
l_mtlr.fit(X_train, T_train, E_train, lr=0.00001, l2_reg=0.001, init_method='orthogonal')

## Evaluate model
tmp_results = evaluate_model(l_mtlr,X_train,T_train,E_train,X_test,T_test,E_test)
tmp_results['model'] = ['l_mtlr']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### Neural MTLR model

In [ ]:
from pysurvival.models.multi_task import NeuralMultiTaskModel
import pandas as pd

#### 4 - Creating an instance of the Neural MTLR model and fitting the data.

# Defining the MLP structure. Here we will build a 1-hidden layer 
# with 150 units and `Swish` as its activation function
structure = [ {'activation': 'ReLU', 'num_units': 150},  ]

# Building the model
n_mtlr = NeuralMultiTaskModel(structure=structure, bins=150)
n_mtlr.fit(X_train, T_train, E_train, 
            lr=0.00001, l2_reg=0.001, num_epochs = 500,
           init_method='orthogonal', optimizer = 'sgd')

## Evaluate model
tmp_results = evaluate_model(n_mtlr,X_train,T_train,E_train,X_test,T_test,E_test)
tmp_results['model'] = ['n_mtlr']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### Parametric Weibull Model

In [ ]:
from pysurvival.models.parametric import WeibullModel
import pandas as pd

# Creating an instance of the weibertz model and fitting the data.

## Build the model
weib = WeibullModel()
weib.fit(X_train, T_train, E_train, lr=0.0001, init_method='zeros',
    optimizer ='adam', l2_reg = 0.001, num_epochs=2000)

## Evaluate model
tmp_results = evaluate_model(weib,X_train,T_train,E_train,X_test,T_test,E_test)
tmp_results['model'] = ['weib']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### Random Survival Forest model

In [ ]:
from pysurvival.models.survival_forest import RandomSurvivalForestModel
import pandas as pd

## Build the model
rsf = RandomSurvivalForestModel(num_trees=50)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

## Evaluate model
tmp_results = evaluate_model(rsf,X_train,T_train,E_train,X_test,T_test,E_test)
tmp_results['model'] = ['rsf']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### Linear SVM model

In [ ]:
from pysurvival.models.svm import LinearSVMModel
import pandas as pd

# Creating an instance of the Linear SVM model and fitting the data.
## Build Model
svm_l = LinearSVMModel()
svm_l.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

## Evaluate model
tmp_results = evaluate_model(svm_l,X_train,T_train,E_train,X_test,T_test,E_test,ibs=False)
tmp_results['model'] = ['svm_l']
results = pd.concat([results, tmp_results], ignore_index=True)
results.head()

### Evaluate Results

In [ ]:
# Show results table

results.head(10)

In [ ]:
print('max c_index is on line:{}'.format(results['c_index'].idxmax()))
print('max mean_auc is on line:{}'.format(results['mean_auc'].idxmax()))
print('min ibs is on line:{}'.format(results['ibs'].idxmin()))